In [1]:
# Priority...

# if changed from code, this is priority, update everything else.
#   elif changed from external, take this and update everything.

# TODO: Change update() to reflect this.

In [2]:
ns = {
    #"test": 0,
}
widgets = {"test": {"value": 5, "min": 0, "max": 20}}

In [3]:
from ipywidgets import IntSlider
from IPython.display import display as _display

def changed_values(old_values, values):
    """
    :return: dict containing any values that have been added or changed.
    """
    changed = {}
    for k, v in values.items():
        if k in old_values and v == old_values[k]:
            continue
        changed[k] = v
    return changed

class IntegerVariable:
    def __init__(self, value, min, max):
        self.value = value
        self.min = min
        self.max = max
        
    def __repr__(self):
        return f"<IntegerVariable {self.value}, min={self.min}, max={self.max}>"

    
VARIABLE_TYPES = {
    int: IntegerVariable,
}

def new_variable_from_value(**kwargs):
    """
    :param value: value of variable.
    :param type: type of new variable (if not specified, type of value is used.)
    :param kwargs: Expanded and passed to variable constructor.
    
    Build variables by looking up the type of specif
    
    >>> new_variable_from_value(value=5, min=0, max=20)
    <IntegerVariable 5, min=0, max=20>
    
    >>> new_variable_from_value(value=5, type=int, min=0, max=20)
    <IntegerVariable 5, min=0, max=20>
    """
    value = kwargs.pop("value")
    _type = kwargs.pop("type", None) or type(value)
    klass = VARIABLE_TYPES.get(_type)
    if klass is None:
       raise ValueError(f"I don't know how to create a variable from a {type(value)}.")
    
    return klass(value, **kwargs)
    
    

        
class IPyWidgetsView:
    """
    View to interact with Shoebot variables as IPython widgets.
    """
    def __init__(self):
        self.widgets = {}
        
    def add_variable(self, name, variable):
        widget = IntSlider(value=variable.value, description=name.capitalize())
        self.widgets[name] = widget
    
    def get_values(self):
        return {name: widget.value for name, widget in self.widgets.items()}
    
    def update_values(self, values):
        """
        :param values: dict of new values.
        """
        for k, v in values.items():
            widget = self.widgets.get(k)
            if widget:
                widget.value = v
                
    def display(self):
        for widget in self.widgets.values():
            _display(widget)

class VariablesModel:
    """
    Model manages storage of Shoebot variables, 
    """
    def __init__(self, values):
        """
        :param values:   Dictionary of variables to manage.
        """
        self.old_values = {}
        self.values = values
        
    def add_variable(self, name, variable):
        self.change_value(name, variable.value)
        return variable

    def get_changed_values(self):
        """
        :return: dict containing any values that have been added or changed.
        """
        return changed_values(self.old_values, self.values)

    def change_value(self, name, value):
        """
        Set value and mark as changed.
        """
        self.old_values.pop(name, None)
        self.values[name] = value

        
class VariablesController:
    """
    Shoebot "variables" use an MVC based system to allow external
    updates from endpoints such as IPython widgets or an HTTP API.
    """
    def __init__(self, ns):
        self.model = VariablesModel(ns)
        self.view = IPyWidgetsView()
        
    def add_variables(self, widget_kwargs):
        variables = {}
        for name, kwargs in widget_kwargs.items():
            print(kwargs)
            variable = new_variable_from_value(**kwargs)            
            self.view.add_variable(name, variable)
            
            self.model.add_variable(name, variable)
            # Model should not trigger an update, so set it's old_value
            self.model.old_values[name] = variable.value
            
            variables[name] = variable
            
        self.view.update_values({name: variable.value for name, variable in variables.items()})
        return variables

    def update(self):
        """
        Sync up model and view.
        """
        old_values = dict(self.model.old_values)
        model_changed = self.model.get_changed_values()
        view_changed = changed_values(self.model.old_values, self.view.get_values())
        
        # start with view data, then overwrite with model
        merged = dict(view_changed, **model_changed)
                
        self.view.update_values(model_changed)
        self.model.values.update(merged)
        return merged


class Context:
    def __init__(self):
        self.ns = {}
        self._variables = VariablesController(self.ns)

    def _add_variables(self, widget_kwargs):
        """
        :param new_values:  These values 
        """
        # Called at beginning of bot script to crea
        return self._variables.add_variables(widget_kwargs)

    def _update_variables(self):
        """
        Update variable in namespace from widgets
        """
        return self._variables.update()


def setup(widgets):
    global ctx
    
    ctx = Context()
    variables = ctx._add_variables(widgets)

In [4]:
# Test setup sets namespace and external widgets.
widgets = {"test": {"value": 5, "min": 0, "max": 20}}

setup(widgets)

ctx._variables.view.display()

assert ctx.ns == {"test": 5}, f'Expected {"test": 5} but found {ctx.ns}'
assert ctx._variables.view.widgets["test"].value == 5, ctx._variables.view.widgets["test"].value

{'value': 5, 'min': 0, 'max': 20}


IntSlider(value=5, description='Test')

In [5]:
# view change
# Test changing the slider value updates model

widgets = {"test": {"value": 8, "min": 0, "max": 20}}

setup(widgets)


# pre-requisites
assert ctx.ns == {"test": 8}, f'Expected {"test": 8} but found {ctx.ns}'
assert ctx._variables.view.widgets["test"].value == 8, ctx._variables.view.widgets["test"].value

ctx._variables.view.widgets["test"].value = 1

ctx._update_variables()

#ctx._variables.view.display()
display(ctx._variables.view.widgets['test'])


assert ctx.ns["test"] == 1
assert ctx._variables.view.widgets["test"].value == 1, ctx._variables.view.widgets["test"].value

{'value': 8, 'min': 0, 'max': 20}


IntSlider(value=1, description='Test')

In [6]:
# Test updating model updates slider value
widgets = {"test": {"value": 3, "min": 0, "max": 20}}

setup(widgets)

# pre-requisites
assert ctx.ns == {"test": 3}, f'Expected {"test": 3} but found {ctx.ns}'
assert ctx._variables.view.widgets["test"].value == 3, ctx._variables.view.widgets["test"].value

ctx.ns["test"] = 10
ctx._update_variables()

ctx._variables.view.display()

print(ctx.ns["test"])
print(ctx._variables.view.widgets)

assert ctx.ns["test"] == 10  # TODO
assert ctx._variables.view.widgets["test"].value == 10

{'value': 3, 'min': 0, 'max': 20}


IntSlider(value=10, description='Test')

10
{'test': IntSlider(value=10, description='Test')}


In [11]:
# If there is a change from both model (code) and view,
# then the model change should take priority.
widgets = {"test": {"value": 5, "min": 0, "max": 20}}

setup(widgets)

# pre-requisites
assert ctx.ns == {"test": 5}, f'Expected {"test": 5} but found {ctx.ns}'
assert ctx._variables.view.widgets["test"].value == 5, ctx._variables.view.widgets["test"].value

ctx._variables.view.widgets["test"].value = 8
ctx.ns["test"] = 7
ctx._update_variables()

ctx._variables.view.display()

print(ctx.ns["test"])
print(ctx._variables.view.widgets)

assert ctx.ns["test"] == 7  # TODO
assert ctx._variables.view.widgets["test"].value == 7

{'value': 5, 'min': 0, 'max': 20}


IntSlider(value=7, description='Test')

7
{'test': IntSlider(value=7, description='Test')}


In [10]:
{"a": 1, "b": {"type": "int", "min": 0, "max": 10, "value": 100}}

{'a': 1, 'b': {'type': 'int', 'min': 0, 'max': 10, 'value': 100}}

In [9]:
# In Shoebot 1.x, you could pass values for variables - but only those that use add_variable in the code.
# For Shoebot 2, we want to set anything in the namespace..

# Option 1:   Pass type info first 
# Option 2:   Run bot frame and get type info...

# Option 3:   Analyse code to find types :)